In [41]:
import numpy as np
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

D:\GEGDC\KT00475829\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
os.chdir("D:\\GEGDC\\KT00475829\\R work\\Data sets\\titanic dataset")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [6]:
#caluculating % of missing values
total = train.isnull().sum().sort_values(ascending = False)
per = (train.isnull().sum()*100)/train.isnull().count()
percent_2 = (round(per, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])

In [7]:
missing_data.head(5)

,Total,%
Cabin,687,77.1
Age,177,19.9
Embarked,2,0.2
Fare,0,0.0
Ticket,0,0.0


In [9]:
type(train.columns.values)

numpy.ndarray

In [10]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
train = train.drop(['PassengerId'], axis=1)

In [16]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
#Extracting deck from cabin feature
import re
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
data = [train, test]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int) 

In [21]:
#dropping cabin feature
train = train.drop(['Cabin'], axis  = 1)
test = test.drop(['Cabin'], axis = 1)

In [22]:
#creating random integers of Age values with mean and standard deviation
data = [train, test]
for dataset in data:
    mean = train["Age"].mean()
    std = test["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # filling NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train["Age"].astype(int)

In [23]:
train["Age"].isnull().sum()

0

In [24]:
type(age_slice)

pandas.core.series.Series

In [26]:
#Filling Embarked with most common value
train['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [27]:
common_value = 'S'
data = [train, test]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(common_value)

In [28]:
#Dropping name column
for dataset in data:
    dataset = dataset.drop(['Name'], axis = 1)

In [29]:
#Converting sex to numeric
genders = {"male": 0, "female": 1}
data = [train, test]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

In [31]:
train['Ticket'].describe()

count        891
unique       681
top       347082
freq           7
Name: Ticket, dtype: object

In [32]:
#Dropping ticket column
train = train.drop(['Ticket'], axis=1)
test = test.drop(['Ticket'], axis=1)

In [33]:
#Converting Embarked into numeric
ports = {"S": 0, "C": 1, "Q": 2}
data = [train, test]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].map(ports)

In [34]:
#Creating age group variable
data = [train, test]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6

In [35]:
train['Age'].value_counts()

6    163
4    159
5    147
3    141
2    115
1     98
0     68
Name: Age, dtype: int64

In [36]:
train.head(15)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
0,0,3,"Braund, Mr. Owen Harris",0,2,1,0,7.2500,0,8
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,5,1,0,71.2833,1,3
2,1,3,"Heikkinen, Miss. Laina",1,3,0,0,7.9250,0,8
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,5,1,0,53.1000,0,3
4,0,3,"Allen, Mr. William Henry",0,5,0,0,8.0500,0,8
5,0,3,"Moran, Mr. James",0,3,0,0,8.4583,2,8
6,0,1,"McCarthy, Mr. Timothy J",0,6,0,0,51.8625,0,5
7,0,3,"Palsson, Master. Gosta Leonard",0,0,3,1,21.0750,0,8
8,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,3,0,2,11.1333,0,8
9,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,1,1,0,30.0708,1,8


In [37]:
test.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
0,892,3,"Kelly, Mr. James",0,2,0,0,7.8292,2,8
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,5,1,0,7.0000,0,8
2,894,2,"Myles, Mr. Thomas Francis",0,3,0,0,9.6875,2,8
3,895,3,"Wirz, Mr. Albert",0,5,0,0,8.6625,0,8
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,5,1,1,12.2875,0,8
5,897,3,"Svensson, Mr. Johan Cervin",0,3,0,0,9.2250,0,8
6,898,3,"Connolly, Miss. Kate",1,6,0,0,7.6292,2,8
7,899,2,"Caldwell, Mr. Albert Francis",0,0,1,1,29.0000,0,8
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",1,3,0,0,7.2292,1,8
9,901,3,"Davies, Mr. John Samuel",0,1,2,0,24.1500,0,8


In [38]:
#dropping name column
train = train.drop(['Name'], axis = 1)
test = test.drop(['Name'], axis = 1)

In [39]:
train.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
0,0,3,0,2,1,0,7.2500,0,8
1,1,1,1,5,1,0,71.2833,1,3
2,1,3,1,3,0,0,7.9250,0,8
3,1,1,1,5,1,0,53.1000,0,3
4,0,3,0,5,0,0,8.0500,0,8
5,0,3,0,3,0,0,8.4583,2,8
6,0,1,0,6,0,0,51.8625,0,5
7,0,3,0,0,3,1,21.0750,0,8
8,1,3,1,3,0,2,11.1333,0,8
9,1,2,1,1,1,0,30.0708,1,8


In [64]:
X_train = train.drop("Survived", axis=1)
Y_train = train["Survived"]
X_test  = test.drop("PassengerId", axis=1).copy()

In [47]:
total = train.isnull().sum().sort_values(ascending = False)
per = (train.isnull().sum()*100)/train.isnull().count()
percent_2 = (round(per, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])

In [48]:
missing_data.head(10)

,Total,%
Deck,0,0.0
Embarked,0,0.0
Fare,0,0.0
Parch,0,0.0
SibSp,0,0.0
Age,0,0.0
Sex,0,0.0
Pclass,0,0.0
Survived,0,0.0


In [49]:
total = test.isnull().sum().sort_values(ascending = False)
per = (test.isnull().sum()*100)/test.isnull().count()
percent_2 = (round(per, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])

In [50]:
missing_data.head(10)

,Total,%
Fare,1,0.2
Deck,0,0.0
Embarked,0,0.0
Parch,0,0.0
SibSp,0,0.0
Age,0,0.0
Sex,0,0.0
Pclass,0,0.0
PassengerId,0,0.0


In [52]:
test.dropna()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
0,892,3,0,2,0,0,7.8292,2,8
1,893,3,1,5,1,0,7.0000,0,8
2,894,2,0,3,0,0,9.6875,2,8
3,895,3,0,5,0,0,8.6625,0,8
4,896,3,1,5,1,1,12.2875,0,8
5,897,3,0,3,0,0,9.2250,0,8
6,898,3,1,6,0,0,7.6292,2,8
7,899,2,0,0,1,1,29.0000,0,8
8,900,3,1,3,0,0,7.2292,1,8
9,901,3,0,1,2,0,24.1500,0,8


In [53]:
total = test.isnull().sum().sort_values(ascending = False)
per = (test.isnull().sum()*100)/test.isnull().count()
percent_2 = (round(per, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])

In [54]:
missing_data.head(10)

,Total,%
Fare,1,0.2
Deck,0,0.0
Embarked,0,0.0
Parch,0,0.0
SibSp,0,0.0
Age,0,0.0
Sex,0,0.0
Pclass,0,0.0
PassengerId,0,0.0


In [56]:
test['Fare'].describe()

count    417.000000
mean      35.627188
std       55.907576
min        0.000000
25%        7.895800
50%       14.454200
75%       31.500000
max      512.329200
Name: Fare, dtype: float64

In [58]:
test = test.dropna()

In [60]:
total = test.isnull().sum().sort_values(ascending = False)
per = (test.isnull().sum()*100)/test.isnull().count()
percent_2 = (round(per, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])

In [61]:
missing_data.head(10)

,Total,%
Deck,0,0.0
Embarked,0,0.0
Fare,0,0.0
Parch,0,0.0
SibSp,0,0.0
Age,0,0.0
Sex,0,0.0
Pclass,0,0.0
PassengerId,0,0.0


In [74]:
# Random Forest
random_forest2 = RandomForestClassifier(n_estimators=100)
random_forest2.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [76]:
X_train.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
0,3,0,2,1,0,7.2500,0,8
1,1,1,5,1,0,71.2833,1,3
2,3,1,3,0,0,7.9250,0,8
3,1,1,5,1,0,53.1000,0,3
4,3,0,5,0,0,8.0500,0,8
5,3,0,3,0,0,8.4583,2,8
6,1,0,6,0,0,51.8625,0,5
7,3,0,0,3,1,21.0750,0,8
8,3,1,3,0,2,11.1333,0,8
9,2,1,1,1,0,30.0708,1,8


In [77]:
Y_train.head(10)

0    0
1    1
2    1
3    1
4    0
5    0
6    0
7    0
8    1
9    1
Name: Survived, dtype: int64

In [78]:
random_forest2

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [79]:
Y_prediction = random_forest2.predict(X_test)

In [80]:
random_forest.score(X_train, Y_train)

0.9562289562289562